## Importación de paquetes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian, acos)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import ReferenceFrame, Point, LagrangesMethod
from sympy import legendre_poly

from functools import lru_cache

## Inicializar espacio de trabajo

In [ ]:
init_vprinting()

In [ ]:
x, t = symbols('x t')

## Probando Polinomios de Legendre

In [ ]:
legendre_poly(20)

In [ ]:
N = 100
prec = 20
for ii in legendre_poly(N, polys = True).real_roots():
    pass
    #print(ii.evalf(n = prec))

In [ ]:
N = 100
prec = 20
for ii in legendre_poly(N, polys = True).diff().real_roots():
    pass
    #print(ii.evalf(n = prec))

## Funciones que generan N collocation points

In [ ]:
@lru_cache
def LG(N, precission = 20):
    return [ii.evalf(n = precission) for ii in legendre_poly(N, polys = True).real_roots()]
@lru_cache
def LGR(N, precission = 20):
    pol = legendre_poly(N, polys = True) + legendre_poly(N-1, polys = True)
    return [ii.evalf(n = precission) for ii in pol.real_roots()]
@lru_cache
def LGL(N, precission = 20):
    root_list = [ii.evalf(n = precission) for ii in legendre_poly(N-1, polys = True).diff().real_roots()]
    return [-1.,] + root_list + [1.,]
@lru_cache
def LGLm(N, precission = 20):
    return LGL(N+2, precission)[1:-1]
@lru_cache
def LG_1(N, precission = 20):
    return [-1] + LG(N-1, precission)
@lru_cache
def LG2(N, precission = 20):
    return [-1]+LG(N-2, precission)+[1]

## Bases de Polinomios de Lagrange (simbólico)

In [ ]:
#### CREDIT: https://gist.github.com/folkertdev/084c53887c49a6248839 ####

from operator import mul
from functools import reduce, lru_cache
from itertools import chain

# sympy symbols
x = symbols('x')

# convenience functions
product = lambda *args: reduce(mul, *(list(args) + [1]))


# this product may be reusable (when creating many functions on the same domain)
# therefore, cache the result
@lru_cache(16)
def lag_pol(labels, j):
    def gen(labels, j):
        k = len(labels)
        current = labels[j]
        for m in labels:
            if m == current:
                continue
            yield (x - m) / (current - m)
    return expand(product(gen(labels, j)))


def lagrangePolynomial(xs, ys):
    # based on https://en.wikipedia.org/wiki/Lagrange_polynomial#Example_1
    k = len(xs)
    total = 0

    # use tuple, needs to be hashable to cache
    xs = tuple(xs)

    for j, current in enumerate(ys):
        t = current * lag_pol(xs, j)
        total += t

    return total

In [ ]:
a, b, c = symbols('a b c')
abcpol = lagrangePolynomial([-1, 0, 1], [a, b, c])
abcpol

In [ ]:
h = symbols('h')

_parab_interp = lagrangePolynomial([0, h/2, h],[a,b,c])
_parab_interp.factor().collect(x)

In [ ]:
@lru_cache
def phi_LGL(N, i):
    return lag_pol(tuple(LGL(N)), i)
@lru_cache
def phi_LG_1(N, i):
    return lag_pol(tuple(LG_1(N)), i)
@lru_cache
def phi_LG2(N, i, precission = 20):
    return lag_pol(tuple(LG2(N, precission)), i)

In [ ]:
jj = phi_LGL(3,0)
jj

In [ ]:
jj.evalf(subs = {x:1})

In [ ]:
def LGL_abcpol(N, precission = 20):
    letters = [symbols(ii) for ii in 'abcdefghijklmnopqrsuvw']
    return lagrangePolynomial(LGL(N, precission), letters[:N])

def LG_1_abcpol(N, precission = 20):
    letters = [symbols(ii) for ii in 'abcdefghijklmnopqrsuvw']
    return lagrangePolynomial(LG_1(N, precission), letters[:N])

def LG2_abcpol(N, precission = 20):
    letters = [symbols(ii) for ii in 'abcdefghijklmnopqrsuvw']
    return lagrangePolynomial(LG2(N, precission), letters[:N])


In [ ]:
@lru_cache
def LG_1_end_p_fun(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.subs(x, 1)
    return lambdify(coefs, res)

@lru_cache
def LG_1_diff_end_p_fun(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.diff(x).subs(x, 1)
    return lambdify(coefs, res)

## Matrices de Derivación y Barycentric coordinates

In [ ]:
@lru_cache
def get_taus(n, scheme, precission = 20):
    if scheme == 'LGL' or scheme == 'D2':
        taus = LGL(n, precission)
    elif scheme == 'LG':
        taus = LG_1(n, precission)
    elif scheme == 'LG2':
        taus = LG2(n, precission)
    elif scheme == 'LGLm':
        taus = LGL(n, precission)
    return taus

In [ ]:
@lru_cache
def v_coef(n, i, scheme = 'LGL', precission = 20):
    taus = get_taus(n+1, scheme, precission)
    prod_coef = [ii for ii in range(n+1)]
    prod_coef.pop(i)
    v_i = 1.0
    for jj in prod_coef:
        v_i *= (taus[i]-taus[jj])
    return 1./v_i

In [ ]:
@lru_cache
def matrix_D_bary(n, scheme = 'LGL', precission = 20):
    taus = get_taus(n+1, scheme, precission)
    M = zeros(n+1)
    v_arr = [v_coef(n, ii, scheme, precission) for ii in range(n+1)]
    for i in range(n+1):
        j_range = [j for j in range(n+1)]
        j_range.pop(i)
        for j in j_range:
            M[i, j] = (v_arr[j]/v_arr[i])/(taus[i]-taus[j])
    for j in range(n+1):
        M[j,j] = -sum(M[j,:])
    return M

In [ ]:
def v_sum(t_arr, i):
    n = len(t_arr)
    prod_coef = [ii for ii in range(n)]
    prod_coef.pop(i)
    v_i = 1.0
    for jj in prod_coef:
        v_i *= (t_arr[i]-t_arr[jj])
    return 1./v_i

def bary_poly(t_arr, y_arr):
    n = len(t_arr)
    v_arr = [v_sum(t_arr, ii) for ii in range(n)]
    sup = 0
    for i in range(n):
        sup+= v_arr[i]*y_arr[i]/(t-t_arr[i])
    inf = 0
    for i in range(n):
        inf+= v_arr[i]/(t-t_arr[i])
    poly_fun = lambdify([t,], sup/inf)
    
    def new_poly(t):
        t = np.array(t, dtype='float64')
        cond_list = [t == t_i for t_i in t_arr]
        func_list = list(y_arr)
        func_list.append(poly_fun)
        return np.piecewise(t, cond_list, func_list)
        
    return new_poly
    

In [ ]:
matrix_D_bary(3,)

In [ ]:
import casadi as cas

In [ ]:
from optibot.casadi import sympy2casadi

In [ ]:
@lru_cache
def LG_1_end_p_fun_cas(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.subs(x, 1)
    x_cas = cas.SX.sym('x', N)
    res_cas = sympy2casadi(res, coefs, cas.vertsplit(x_cas))
    return cas.Function(
    "dynamics_x",
    [x_cas],
    [res_cas])

@lru_cache
def LG_1_diff_end_p_fun_cas(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.diff(x).subs(x, 1)
    x_cas = cas.SX.sym('x', N)
    res_cas = sympy2casadi(res, coefs, cas.vertsplit(x_cas))
    return cas.Function(
    "dynamics_x",
    [x_cas],
    [res_cas])

In [ ]:
from optibot.pseudospectral import LG_end_p_fun_cas, LG_diff_end_p_fun_cas

## Problema de ejemplo: bloque que desliza

$$\dot x_1 = x_2$$
$$\dot x_2 = u $$

Minimizar $t_f$ sujeto a:

$t_0 = 0\$

$x_2(0) = x_2(t_f) = 0$

$x_1(0) = 0\ ,\ x_1(t_f) = 1$

$ u_{min} < u(t) < u_{max}$

In [ ]:
LG_1_end_p_fun(3, 20)(*np.array([1,2,3]))

In [ ]:
from scipy.interpolate import CubicHermiteSpline as hermite

@lru_cache
def get_taus(n, scheme, precission = 20):
    if scheme == 'LGL' or scheme == 'D2':
        taus = LGL(n, precission)
    elif scheme == 'LG':
        taus = LG_1(n, precission)
    elif scheme == 'LG2':
        taus = LG2(n, precission)
    elif scheme == 'LGLm':
        taus = LGL(n, precission)
    return taus

def find_der_polyline(x_n, xp, yp):
    n = np.searchsorted(xp, x_n)
    n = np.where(n-1>0, n-1, 0)
    deriv_arr = (yp[1:] - yp[:-1])/(xp[1:] - xp[:-1])
    return deriv_arr[n]

def get_pol_u(scheme, N, uu): 
    if scheme == 'LG2':
        pol_u = bary_poly(LG(N-2), uu)
    elif scheme == 'LG':
        pol_u = bary_poly(LG(N-1), uu)
    elif scheme == 'LGLm':
        pol_u = bary_poly(LGLm(N-2), uu)
    elif scheme == 'LGL' or 'D2':
        pol_u = bary_poly(LGL(N), uu)
    return pol_u

def get_pol_x(scheme, qq, vv, t0, t1):
    N = len(qq)
    tau_x = get_taus(N, scheme)
    qq_d = 2/(t1 - t0) * matrix_D_bary(N-1, scheme)@qq
    vv_d = 2/(t1 - t0) * matrix_D_bary(N-1, scheme)@vv

    pol_q = bary_poly(tau_x, qq)
    pol_v = bary_poly(tau_x, vv)
    pol_q_d = bary_poly(tau_x, qq_d)
    pol_v_d = bary_poly(tau_x, vv_d)
    return pol_q, pol_v, pol_q_d, pol_v_d

def extend_x_arrays(qq, vv, scheme):
    N = len(qq)
    if scheme == 'LG':
        tau_x = LG_1(N) + [1]
        endp_f = LG_1_end_p_fun(N, 20)
        qq_1 = float(endp_f(*qq))
        vv_1 = float(endp_f(*vv))
        qq = np.array(list(qq) + [qq_1,],dtype='float64')
        vv = np.array(list(vv) + [vv_1,],dtype='float64')
    else:
        tau_x = get_taus(N, scheme)
    return tau_x, qq, vv

def extend_u_array(uu, scheme, N):
    tau_u = get_taus(N, scheme)
    if scheme == 'LG2':
        uu = np.array([uu[0]] + list(uu) + [uu[-1]],dtype='float64')
    elif scheme == 'LG':
        tau_u = LG_1(N) + [1]
        uu = np.array([uu[0]] + list(uu) + [uu[-1]],dtype='float64')
    elif scheme == 'LGLm':
        uu = np.array([uu[0]] + list(uu) + [uu[-1]])
    return tau_u, uu

def get_hermite_x(qq, vv, aa, tau_x, t0, t1):
    N = len(qq)
    coll_p = t_0 + (1 + np.array(tau_x,dtype='float64'))*(t1 - t0)/2
    her_q = hermite(coll_p, qq, vv)
    her_v = hermite(coll_p, vv, aa)
    her_q_d = her_q.derivative()
    her_v_d = her_v.derivative()
    return her_q, her_v, her_q_d, her_v_d

def error_transcr(qq, vv, uu, scheme, t0, t1, u_interp = 'pol', x_interp = 'pol', problem='bloq'):
    N = len(qq)
    scheme_opts = ['LG', 'LGL', 'D2', 'LG2', 'LGLm']
    if scheme not in scheme_opts:
        NameError(f'Invalid scheme.\n valid options are {scheme_opts}')
    t_arr = np.linspace(-1, 1, 1000)
    if u_interp == 'pol':
        pol_u = get_pol_u(scheme, N, uu)
        u_arr = pol_u(t_arr)
    elif u_interp == 'lin':
        tau_u, uu = extend_u_array(uu, scheme, N)
        u_arr = np.interp(t_arr, tau_u, uu)
    elif u_interp == 'smooth':
        tau_u, uu = extend_u_array(uu, scheme, N)
        uu_dot = np.gradient(uu, tau_u)
        u_arr = hermite(tau_u, uu, uu_dot)(t_arr)
    else:
        raise NameError('Invalid interpolation method for u.\n valid options are "pol", "lin", "smooth"') 
        
    if x_interp == 'pol':
        tau_x = get_taus(N, scheme)
        pol_q, pol_v, pol_q_d, pol_v_d = get_pol_x(scheme, qq, vv, t0, t1)
        q_arr = pol_q(t_arr)
        v_arr = pol_v(t_arr)
        q_arr_d = pol_q_d(t_arr)
        v_arr_d = pol_v_d(t_arr)
    elif x_interp == 'lin':
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        q_arr = np.interp(t_arr, tau_x, qq)
        v_arr = np.interp(t_arr, tau_x, vv)
        coll_p = t_0 + (1 + np.array(tau_x,dtype='float64'))*(t1 - t0)/2
        t_arr_lin = np.linspace(t0, t1, 1000)
        q_arr_d = find_der_polyline(t_arr_lin, coll_p, qq)
        v_arr_d = find_der_polyline(t_arr_lin, coll_p, vv)
    elif x_interp == 'Hermite':
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        if problem == 'bloq':
            aa = uu
        elif problem == 'pend':
            aa = uu - np.sin(qq)
        her_q, her_v, her_q_d, her_v_d = get_hermite_x(qq, vv, aa, tau_x, t0, t1)
        t_arr_lin = np.linspace(t0, t1, 1000)
        q_arr = her_q(t_arr_lin)
        v_arr = her_v(t_arr_lin)
        q_arr_d = her_q_d(t_arr_lin)
        v_arr_d = her_v_d(t_arr_lin)
    else:
        raise NameError('Invalid interpolation method for x.\n valid options are "pol", "lin", "Hermite"') 
    
    if problem == 'bloq':
        err_q = (q_arr_d-v_arr)**2
        err_v = (v_arr_d-u_arr)**2
    elif problem == 'pend':
        err_q = (q_arr_d-v_arr)**2
        err_v = (v_arr_d-(u_arr - np.sin(q_arr)))**2
    else:
        raise NameError('Invalid problem name') 
    
    err_q_num = np.trapz(err_q, t_arr)* (t1-t0)/2
    err_v_num = np.trapz(err_v, t_arr)* (t1-t0)/2
    
    
    return err_q_num, err_v_num, err_q_num + err_v_num 

In [ ]:
import time

In [ ]:
solve_repetitions = 30

#### LGL: función objetivo alternativa

$u(\tau)$ está definida como el polinomio de grado N-1 que pasa por los N valores calculados de $U^N$. Encontrar cómo obtener $\int_{-1}^{1}{u(\tau)^2}$ en función de $U^N$

In [ ]:
def obj_f_u_LGL(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LGL(N, precission), coefs)
    res = integrate(pol_lag ** 2, [x, -1, 1])
    return lambdify(coefs, res)

def obj_f_u_LG2(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG(N, precission), coefs)
    res = integrate(pol_lag ** 2, [x, -1, 1])
    return lambdify(coefs, res)

### Formulación clásica $\dot x = f(x,u)$

In [ ]:
x_sym = cas.SX.sym('x', 2)
x_dot_sym = cas.SX.sym('x_dot', 2)
u_sym = cas.SX.sym('u',1)

dynam_f_x = cas.Function(
    "dynamics_x",
    [x_sym, x_dot_sym, u_sym],
    [x_dot_sym[1]-u_sym]
)

In [ ]:
N = 7
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
x_opti = opti.variable(N, 2)
u_opti = opti.variable(N)
tau_arr = LGL(N, 20)
t_f = opti.variable(1)
t_0 = 0
u_lim = opti.parameter(1)

In [ ]:
D_mat = sympy2casadi(matrix_D_bary(N-1, 'LGL',20), [], [])
x_dot_opti = (2/t_f - t_0) * D_mat@x_opti

#cost = cas.sum1(u_opti**2)
#f_obj = obj_f_u_LGL(N)
#cost = f_obj(*cas.vertsplit(u_opti))
cost = t_f
opti.minimize(cost)

In [ ]:
opti.subject_to(x_opti[0,:].T == [0., 0.])
opti.subject_to(x_opti[-1,:].T == [1. ,0.])
opti.subject_to(t_f > 0)

for ii in range(N):
    opti.subject_to(dynam_f_x(x_opti[ii,:], x_dot_opti[ii,:], u_opti[ii,:])==0)
    opti.subject_to(x_opti[ii,1] == x_dot_opti[ii,0])
    opti.subject_to(u_opti[ii] <= u_lim[0])
    opti.subject_to(-u_lim[0] <= u_opti[ii])

opti.set_value(u_lim, 1.)
opti.set_initial(t_f, 1.)

In [ ]:
cput0 = time.time()
for ii in range(solve_repetitions):
    sol = opti.solve()
cput1 = time.time()
cpudt_LGL = (cput1-cput0)/solve_repetitions

In [ ]:
xx_sol_LGL = sol.value(x_opti)
uu_sol_LGL = sol.value(u_opti)
t_f_LGL = sol.value(t_f)

t_arr_LGL = t_0 + (1 + np.array(tau_arr))*(t_f_LGL - t_0)/2

q_pol_LGL = lagrangePolynomial(t_arr_LGL, xx_sol_LGL[:,0])
v_pol_LGL = lagrangePolynomial(t_arr_LGL, xx_sol_LGL[:,1])
u_pol_LGL = lagrangePolynomial(t_arr_LGL, uu_sol_LGL[:])

q_interp_f_LGL = lambdify([x,],q_pol_LGL)
v_interp_f_LGL = lambdify([x,],v_pol_LGL)
u_interp_f_LGL = lambdify([x,],u_pol_LGL)

In [ ]:
t_arr_LGL

In [ ]:
plt.figure(figsize=[12,10])
plt.plot(t_arr_LGL, xx_sol_LGL[:,0], 'ro', label = 'position')
plt.plot(t_arr_LGL, xx_sol_LGL[:,1], 'bo', label = 'speed')
plt.plot(t_arr_LGL, uu_sol_LGL[:], 'go', label = 'control')
N_interp = 300
t_int_arr_LGL = np.linspace(t_0, t_f_LGL, N_interp)
plt.plot(t_int_arr_LGL, q_interp_f_LGL(t_int_arr_LGL), 'r')
plt.plot(t_int_arr_LGL, v_interp_f_LGL(t_int_arr_LGL), 'b')
plt.plot(t_int_arr_LGL, u_interp_f_LGL(t_int_arr_LGL), 'g')
plt.grid()
plt.legend()

In [ ]:
#u_sq_LGL = u_interp_f_LGL(t_int_arr)**2
#quality_LGL = np.trapz(u_sq_LGL, t_int_arr)
quality_LGL = t_f_LGL
quality_LGL

In [ ]:
cpudt_LGL*1000

trun_err_LGL = error_transcr(q_pol_LGL, v_pol_LGL, u_pol_LGL, t_0, t_f_LGL)
trun_err_LGL

### Formulación $D^2$ de Ross, Rea y Fahroo

In [ ]:
q_sym = cas.SX.sym('q', 1)
q_dot_sym = cas.SX.sym('q_dot', 1)
q_dot_dot_sym = cas.SX.sym('q_dot_dot', 1)
u_sym = cas.SX.sym('u',1)

dynam_f_q = cas.Function(
    "dynamics",
    [q_sym, q_dot_sym, q_dot_dot_sym, u_sym],
    [q_dot_dot_sym-u_sym]
)

In [ ]:
N = 9
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
x_opti = opti.variable(N)
u_opti = opti.variable(N)
tau_arr = LGL(N, 20)
t_f = opti.variable(1)
t_0 = 0
u_lim = opti.parameter(1)

In [ ]:
D_mat = sympy2casadi(matrix_D_bary(N-1, 'D2',20), [], [])
x_dot_opti = (2/t_f - t_0) * D_mat@x_opti
x_dot_dot_opti = (2/t_f - t_0) * D_mat@x_dot_opti

#cost = cas.sum1(u_opti**2)
#f_obj = obj_f_u_LGL(N)
#cost = f_obj(*cas.vertsplit(u_opti))
cost = t_f
opti.minimize(cost)

In [ ]:
opti.subject_to(x_opti[0] == [0.])
opti.subject_to(x_opti[-1] == [1.])
opti.subject_to(x_dot_opti[0] == [0.])
opti.subject_to(x_dot_opti[-1] == [0.])
opti.subject_to(t_f > 0)

for ii in range(N):
    opti.subject_to(dynam_f_q(x_opti[ii], x_dot_opti[ii], x_dot_dot_opti[ii], u_opti[ii,:])==0)
    opti.subject_to(u_opti[ii] <= u_lim[0])
    opti.subject_to(-u_lim[0] <= u_opti[ii])

opti.set_value(u_lim, 1.)
opti.set_initial(t_f, 1.)

In [ ]:
cput0 = time.time()
for ii in range(solve_repetitions):
    sol = opti.solve()
cput1 = time.time()
cpudt_D2 = (cput1-cput0)/solve_repetitions

In [ ]:
xx_sol_D2 = sol.value(x_opti)
uu_sol_D2 = sol.value(u_opti)
vv_sol_D2 = sol.value(x_dot_opti)
t_f_D2 = sol.value(t_f)

t_arr_D2 = t_0 + (1 + np.array(tau_arr))*(t_f_D2 - t_0)/2

q_pol_D2 = lagrangePolynomial(t_arr_D2, xx_sol_D2[:])
v_pol_D2 = lagrangePolynomial(t_arr_D2, xx_sol_D2[:]).diff(x)
u_pol_D2 = lagrangePolynomial(t_arr_D2, uu_sol_D2[:])

q_interp_f_D2 = lambdify([x,],q_pol_D2)
v_interp_f_D2 = lambdify([x,],v_pol_D2)
u_interp_f_D2 = lambdify([x,],u_pol_D2)

In [ ]:
plt.figure(figsize=[12,10])
plt.plot(t_arr_D2, xx_sol_D2[:], 'ro', label = 'position')
plt.plot(t_arr_D2, vv_sol_D2[:], 'bo', label = 'speed')
plt.plot(t_arr_D2, uu_sol_D2[:], 'go', label = 'control')
N_interp = 300
t_int_arr_D2 = np.linspace(t_0, t_f_D2, N_interp)
plt.plot(t_int_arr_D2, q_interp_f_D2(t_int_arr_D2), 'r')
plt.plot(t_int_arr_D2, v_interp_f_D2(t_int_arr_D2), 'b')
plt.plot(t_int_arr_D2, u_interp_f_D2(t_int_arr_D2), 'g')
plt.grid()
plt.legend()

In [ ]:
#u_sq_D2 = u_interp_f_D2(t_int_arr)**2
#quality_D2 = np.trapz(u_sq_D2, t_int_arr)
quality_D2 = t_f_D2
quality_D2

In [ ]:
cpudt_D2*1000

trun_err_D2 = error_transcr(q_pol_D2, v_pol_D2, u_pol_D2, t_0, t_f_D2)
trun_err_D2

## Nuevo esquema LG2

In [ ]:
q_sym = cas.SX.sym('q', 1)
q_dot_sym = cas.SX.sym('q_dot', 1)
q_dot_dot_sym = cas.SX.sym('q_dot_dot', 1)
u_sym = cas.SX.sym('u',1)

dynam_f = cas.Function(
    "dynamics",
    [q_sym, q_dot_sym, q_dot_dot_sym, u_sym],
    [q_dot_dot_sym-u_sym]
)

In [ ]:
N = 7
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
x_opti = opti.variable(N+2)
u_opti = opti.variable(N)
tau_arr_LG2 = LG2(N+2, 20)
t_f = opti.variable(1)
t_0 = 0
u_lim = opti.parameter(1)

In [ ]:
D_mat = sympy2casadi(matrix_D_bary(N+1, "LG2",20), [], [])
x_dot_opti = (2/t_f - t_0) * D_mat@x_opti
x_dot_dot_opti = (2/t_f - t_0) * D_mat@x_dot_opti

#cost = cas.sum1(u_opti**2)
#f_obj = obj_f_u_LG2(N)
#cost = f_obj(*cas.vertsplit(u_opti))
cost = t_f
opti.minimize(cost)

In [ ]:
opti.subject_to(x_opti[0] == [0.])
opti.subject_to(x_opti[-1] == [1.])
opti.subject_to(x_dot_opti[0] == [0.])
opti.subject_to(x_dot_opti[-1] == [0.])
opti.subject_to(t_f > 0)

for ii in range(1, N+1):
    opti.subject_to(dynam_f_q(x_opti[ii], x_dot_opti[ii], x_dot_dot_opti[ii], u_opti[ii-1,:])==0)
    opti.subject_to(u_opti[ii-1] <= u_lim[0])
    opti.subject_to(-u_lim[0] <= u_opti[ii-1])

opti.set_value(u_lim, 1.)
opti.set_initial(t_f, 1.)

In [ ]:
cput0 = time.time()
for ii in range(solve_repetitions):
    sol = opti.solve()
cput1 = time.time()
cpudt_LG2 = (cput1-cput0)/solve_repetitions

In [ ]:
xx_sol_LG2 = sol.value(x_opti)
uu_sol_LG2 = sol.value(u_opti)
vv_sol_LG2 = sol.value(x_dot_opti)
t_f_LG2 = sol.value(t_f)

t_arr_LG2 = t_0 + (1 + np.array(tau_arr_LG2))*(t_f_D2 - t_0)/2
coll_p_t = t_0 + (1 + np.array(LG(N)))*(t_f_LG2 - t_0)/2

q_pol_LG2 = lagrangePolynomial(t_arr_LG2, xx_sol_LG2[:])
v_pol_LG2 = lagrangePolynomial(t_arr_LG2, xx_sol_LG2[:]).diff(x)
u_pol_LG2 = lagrangePolynomial(coll_p_t, uu_sol_LG2[:])

q_interp_f_LG2 = lambdify([x,],q_pol_LG2)
v_interp_f_LG2 = lambdify([x,],v_pol_LG2)
u_interp_f_LG2 = lambdify([x,],u_pol_LG2)

In [ ]:
plt.figure(figsize=[12,10])
plt.plot(t_arr_LG2, xx_sol_LG2[:], 'ro', label = 'position')
plt.plot(t_arr_LG2, vv_sol_LG2[:], 'bo', label = 'speed')
plt.plot(coll_p_t, uu_sol_LG2[:], 'go', label = 'control')
N_interp = 300
t_int_arr_LG2 = np.linspace(t_0, t_f_LG2, N_interp)
plt.plot(t_int_arr_LG2, q_interp_f_LG2(t_int_arr_LG2), 'r')
plt.plot(t_int_arr_LG2, v_interp_f_LG2(t_int_arr_LG2), 'b')
plt.plot(t_int_arr_LG2, u_interp_f_LG2(t_int_arr_LG2), 'g')
plt.grid()
plt.legend()

In [ ]:
#u_sq_LG2 = u_interp_f_LG2(t_int_arr)**2
#quality_LG2 = np.trapz(u_sq_LG2, t_int_arr)
quality_LG2 = t_f_LG2
quality_LG2

In [ ]:
cpudt_LG2*1000

trun_err_LG2 = error_transcr(q_pol_LG2, v_pol_LG2, u_pol_LG2, t_0, t_f_LG2)
trun_err_LG2

## COMPARACIÓN DE RESULTADOS

from IPython.display import Markdown as md
md(f'''| | Usual LGL | $D^2$ | Nuevo LG2 |
| --- | --- | --- | --- |
| Coste a optimizar ($t_f$) | {round(quality_LGL,5)} | {round(quality_D2,5)} | {round(quality_LG2,5)} |
| Valor medio de $(U^N)^2$ | {round(np.mean(uu_sol_LGL**2),4)} | {round(np.mean(uu_sol_D2**2),4)} | {round(np.mean(uu_sol_LG2**2),4)} |
| CPU time (ms) | {round(cpudt_LGL*1000,4)} | {round(cpudt_D2*1000,4)} | {round(cpudt_LG2*1000,4)} |
| Integrated Truncation Error | {"{:.2e}".format(trun_err_LGL)} | {trun_err_D2} | {trun_err_LG2} |''')

In [ ]:
plt.figure(figsize=[16,14])

plt.plot(coll_p_t, uu_sol_LG2[:], 'go', label = 'control LG2')
plt.plot(t_arr_D2, uu_sol_D2[:], 'bo', label = 'control D2')
plt.plot(t_arr_LGL, uu_sol_LGL[:], 'ro', label = 'control LGL')

plt.plot(t_int_arr_LG2, u_interp_f_LG2(t_int_arr_LG2), 'g')
plt.plot(t_int_arr_D2, u_interp_f_D2(t_int_arr_D2), 'b')
plt.plot(t_int_arr_LGL, u_interp_f_LGL(t_int_arr_LGL), 'r--')
plt.plot([0,1,1,2], [1,1,-1,-1], 'k:', label = 'ideal')
plt.vlines([2, t_f_LGL, t_f_D2, t_f_LG2], -1.1, 0.1, colors=['gray', 'r', 'b', 'g'] )
plt.grid()
plt.legend()

## Benchmarking de N y Convergencia

def opt_LGL_bloq_u(N):
    u_arr = np.zeros(N)
    u_arr[:N//2] = 1
    u_arr[-(N//2):] = -1
    return u_arr

In [ ]:
def opti_setup(col_points = 7, scheme = 'LGL', precission = 20, M = 1, U_dim = 1):
    opti = cas.Opti()
    opts = {'ipopt.print_level':0, 'print_time':0}
    opti.solver('ipopt', opts)
    
    opt_dict = {
        'LGL': [col_points, LGL,],
        'D2': [col_points, LGL,],
        'LG2': [col_points+2, LG2,],
        'LGLm': [col_points+2, LGLm,],
        'LG': [col_points+1, LG_1,]
    }
    sch_options = opt_dict[scheme]
    N = sch_options[0]
    
    if scheme == 'LGL' or scheme == 'LG':
        x_opti = opti.variable(N, 2*M)
    else:
        x_opti = opti.variable(N, M)
    
    u_opti = opti.variable(col_points, U_dim)
    tau_arr = sch_options[1](N, precission)
    return opti, N, x_opti, u_opti, tau_arr

def opti_start_point_constr(opti, scheme, x_opti, x_dot_opti, x_s, M=1):
    if scheme == 'LGL':
        opti.subject_to(x_opti[0,:].T == x_s)
    elif scheme == 'LG':
        opti.subject_to(x_opti[0,:].T == x_s)
    elif scheme == 'D2':
        opti.subject_to(x_opti[0,:] == x_s[:M])
        opti.subject_to(x_dot_opti[0,:] == x_s[M:])
    elif scheme == 'LG2':
        opti.subject_to(x_opti[0,:] == x_s[:M])
        opti.subject_to(x_dot_opti[0,:] == x_s[M:])
    elif scheme == 'LGLm':
        opti.subject_to(x_opti[0,:] == x_s[:M])
        opti.subject_to(x_dot_opti[0,:] == x_s[M:])
        
def opti_end_point_constr(opti, scheme, x_opti, x_dot_opti, N, x_f, precission = 20, M=1):
    if scheme == 'LGL':
        opti.subject_to(x_opti[-1,:].T == x_f)
    elif scheme == 'LG':
        endp_f = LG_1_end_p_fun_cas(N, precission)
        for ii in range(2*M):
            opti.subject_to(endp_f(x_opti[:,ii]) == x_f[ii])
    elif scheme == 'D2':
        opti.subject_to(x_opti[-1,:] == x_f[:M])
        opti.subject_to(x_dot_opti[-1,:] == x_f[M:])
    elif scheme == 'LG2':
        opti.subject_to(x_opti[-1,:] == x_f[:M])
        opti.subject_to(x_dot_opti[-1,:] == x_f[M:])
    elif scheme == 'LGLm':
        opti.subject_to(x_opti[-1,:] == x_f[:M])
        opti.subject_to(x_dot_opti[-1,:] == x_f[M:])
        
def opti_col_point_constr(opti, scheme, x_opti, x_dot_opti, x_dot_dot_opti, u_opti, 
                          dynam_f_x, dynam_f_q, N, M=1):
    if scheme == 'LGL':
        for ii in range(N):
            opti.subject_to(dynam_f_x(x_opti[ii,:], x_dot_opti[ii,:], u_opti[ii,:])==0)
            opti.subject_to(x_opti[ii,M:] == x_dot_opti[ii,:M]) 
    if scheme == 'LG':
        #opti.subject_to(x_opti[0,1] == x_dot_opti[0,0]) #convert starting into semicollocation point
        for ii in range(1, N):
            opti.subject_to(dynam_f_x(x_opti[ii,:], x_dot_opti[ii,:], u_opti[ii-1,:])==0)
            opti.subject_to(x_opti[ii,M:] == x_dot_opti[ii,:M])
    elif scheme == 'D2':
        for ii in range(N):
            opti.subject_to(dynam_f_q(x_opti[ii,:], x_dot_opti[ii,:], x_dot_dot_opti[ii,:], u_opti[ii,:])==0)
    elif scheme == 'LG2':
        for ii in range(1, N-1):
            opti.subject_to(dynam_f_q(x_opti[ii,:], x_dot_opti[ii,:], x_dot_dot_opti[ii,:], u_opti[ii-1,:])==0)
    elif scheme == 'LGLm':
        for ii in range(1, N-1):
            opti.subject_to(dynam_f_q(x_opti[ii,:], x_dot_opti[ii,:], x_dot_dot_opti[ii,:], u_opti[ii-1,:])==0)

def chrono_solve(opti, solve_repetitions):
    cput0 = time.time()
    for ii in range(solve_repetitions):
        sol = opti.solve()
    cput1 = time.time()
    cpudt = (cput1-cput0)/solve_repetitions
    return sol, cpudt

def chrono_solve_random_init(opti, solve_repetitions, u_opti, u_lim):
    cpudt = 0.
    err_count = 0
    if not (type(u_lim) == int or type(u_lim) == float):
        u_lim = opti.value(u_lim)
    for ii in range(solve_repetitions):
        
        unresolved = True
        while unresolved:
            try:
                rnd = 2 * u_lim * (np.random.rand(*u_opti.shape)-0.5)
                #print(u_opti, cas.MX(rnd))
                opti.set_initial(u_opti, rnd)
                cput0 = time.time()
                sol = opti.solve()
                cput1 = time.time()
            except RuntimeError:
                print('Runtime error detected, restarting problem')
                err_count += 1
            else:
                unresolved = False
        cpudt += (cput1-cput0)
    cpudt = cpudt/solve_repetitions
    return sol, cpudt, err_count

def opti_read_sol_arrays(scheme, sol, x_opti, u_opti, D_mat, t_f):
    xx_sol = sol.value(x_opti)
    uu_sol = sol.value(u_opti)
    D_mat = np.array(D_mat)
    if scheme == 'LGL':
        qq_sol = xx_sol[:,0]
        vv_sol = xx_sol[:,1]
    elif scheme == 'LG':
        qq_sol = xx_sol[:,0]
        vv_sol = xx_sol[:,1]
    elif scheme == 'D2':
        qq_sol = xx_sol
        vv_sol = 2/(t_f - t_0) * D_mat@qq_sol
    elif scheme == 'LG2':
        qq_sol = xx_sol
        vv_sol = 2/(t_f - t_0) * D_mat@qq_sol
    elif scheme == 'LGLm':
        qq_sol = xx_sol
        vv_sol = 2/(t_f - t_0) * D_mat@qq_sol
    return qq_sol, vv_sol, uu_sol

@lru_cache
def min_t_bloq(col_points = 7, scheme = 'LGL', solve_repetitions = 20, precission = 20):
    opti, N, x_opti, u_opti, tau_arr = opti_setup(col_points, scheme, precission)
    
    t_f = opti.variable(1)
    t_0 = 0
    u_lim = opti.parameter(1)
    
    D_mat = sympy2casadi(matrix_D_bary(N-1, scheme,precission), [], [])
    x_dot_opti = 2/(t_f - t_0) * D_mat@x_opti
    x_dot_dot_opti = 2/(t_f - t_0) * D_mat@x_dot_opti

    cost = t_f
    opti.minimize(cost)
         
    opti.subject_to(t_f > 0)
    opti_start_point_constr(opti, scheme, x_opti, x_dot_opti, x_s = [0,0])
    opti_end_point_constr(opti, scheme, x_opti, x_dot_opti, N, x_f = [1,0])
    opti_col_point_constr(opti, scheme, x_opti, x_dot_opti, x_dot_dot_opti, u_opti, 
                          dynam_f_x, dynam_f_q, N)
    
    for ii in range(col_points):
        opti.subject_to(u_opti[ii,:] <= u_lim[:])
        opti.subject_to(-u_lim[:] <= u_opti[ii,:])

    opti.set_value(u_lim, 1.)
    opti.set_initial(t_f, 1.)
    
    opti.set_initial(u_opti, np.linspace(1,-1,col_points))
    
    sol, cpudt, err_count = chrono_solve_random_init(opti, solve_repetitions, u_opti, u_lim)
    
    t_f_sol = sol.value(t_f)
    
    qq_sol, vv_sol, uu_sol = opti_read_sol_arrays(scheme, sol, x_opti, u_opti, D_mat, t_f_sol)
        
    tran_err_q, tran_err_v, tran_err = error_transcr(qq_sol, vv_sol, uu_sol, scheme, t_0, t_f_sol,
                             u_interp='pol', x_interp='pol', problem='bloq',)
    
    return t_f_sol, cpudt, uu_sol, qq_sol, vv_sol, tran_err_q, tran_err_v, tran_err, err_count

In [ ]:
if not 'results_bloq' in globals():
    results_bloq = {}
test_N = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,30,35,40,45,50]
for scheme in ['LG','LGL', 'D2', 'LG2']:#, 'LGLm'
    print(f'\n\t\t NUEVO ESQUEMA: {scheme}\n')
    t_f_list = []
    cpudt_list = []
    uu_sol_list = []
    qq_sol_list = []
    vv_sol_list = []
    tran_err_q_list = []
    tran_err_v_list = []
    tran_err_list = []
    err_count_list = []
    for N_col in test_N:
        print(f' Problema: {N_col} Collocation Points')
        time.sleep(0.01)
        t_f_sol, cpudt, uu_sol, qq_sol, vv_sol, tran_err_q, tran_err_v, tran_err, err_count = min_t_bloq(col_points = N_col,
                                              scheme = scheme,
                                              solve_repetitions = 300,
                                              precission = 20)
        t_f_list.append(t_f_sol)
        cpudt_list.append(cpudt)
        uu_sol_list.append(uu_sol)
        qq_sol_list.append(qq_sol)
        vv_sol_list.append(vv_sol)
        tran_err_q_list.append(tran_err_q)
        tran_err_v_list.append(tran_err_v)
        tran_err_list.append(tran_err)
        err_count_list.append(err_count)
    results_bloq[scheme] = {
        't_f': t_f_list ,
        'cpu_t' : cpudt_list ,
        'u' : uu_sol_list,
        'N' : test_N,
        'q' : qq_sol_list,
        'v' : vv_sol_list,
        'tr_err_q' : tran_err_q_list,
        'tr_err_v' : tran_err_v_list,
        'tr_err' : tran_err_list,
        'err_count' : err_count_list,}

In [ ]:
plt.figure(figsize=[16,14])

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
plt.title('t_f')
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['t_f'], color + 'o', label = scheme)
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['t_f'], color)
plt.grid()
plt.legend()
plt.vlines(results_bloq['LGL']['N'], 1, 3, 'k',':')
plt.ylim([1.92,2.15])

In [ ]:
plt.figure(figsize=[16,14])

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
plt.title('(t_f - t_f_i)^2')
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], np.abs(np.array(results_bloq[scheme]['t_f'])-2), color + 'o', label = scheme)
    plt.plot(results_bloq[scheme]['N'], np.abs(np.array(results_bloq[scheme]['t_f'])-2), color)
plt.grid(which='both')
plt.legend()
plt.vlines(results_bloq['LGL']['N'], 0, 1, 'k',':')
#plt.ylim([-0.1,0.5])
plt.yscale('log')

In [ ]:
for scheme in schemes:
    print('Total errors', scheme, ': \t', np.sum(results_bloq[scheme]['err_count']))
plt.figure(figsize=[16,14])
plt.title('Errors')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['err_count'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_pend['LGL']['N'], 0, 40, 'k',':')
#plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('CPU time')


schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_bloq['LGL']['N'], 0, 0.08, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('CPU time')


schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_bloq['LGL']['N'], 0, 0.08, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('CPU time')


schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_bloq['LGL']['N'], 0, 0.08, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('CPU time')


schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_bloq['LGL']['N'], 0, 0.08, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Transcription Error')

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err'], color, label = scheme)
plt.grid()
plt.legend()
plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Transcription Error Q')

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_q'], color, label = scheme)
plt.grid()
plt.legend()
plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Transcription Error Dinamics')

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_v'], color, label = scheme)
plt.grid()
plt.legend()
plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
plt.title('Diferencia en t_f entre LG2 y LG')

scheme = schemes[ii]
color = colours[1]
plt.plot(results_bloq[scheme]['N'][:-3], np.abs(np.array(results_bloq['LG']['t_f'])[:-3]-np.array(results_bloq['LG2']['t_f'])[:-3]), color + 'o', label = '$|t_{f, LG}- t_{f, LG2}|$')
plt.plot(results_bloq[scheme]['N'][:-3], np.abs(np.array(results_bloq['LG']['t_f'])[:-3]-np.array(results_bloq['LG2']['t_f'])[:-3]), color)
plt.grid()
plt.legend()
#plt.vlines(results['LGL'][4], 1, 3, 'k',':')
#plt.ylim([1.92,2.15])
plt.yscale('log')

In [ ]:
def poly_u(uu, scheme):
    N = len(uu)
    if scheme == 'LG2':
        tau = LG(N, 30)
    elif scheme == 'LGLm':
        tau = LGLm(N, 30)
    elif scheme == 'LG':
        tau = LG(N, 30)
    else:
        tau = LGL(N, 30)
    return lagrangePolynomial(tau, uu)

In [ ]:
def plot_u(uu, scheme, tf, color = 'k', t0 = 0, N_interp = 300):
    N = len(uu)
    if scheme == 'LG2':
        tau = LG(N, 30)
    elif scheme == 'LGLm':
        tau = LGLm(N, 30)
    elif scheme == 'LG':
        tau = LG(N, 30)
    else:
        tau = LGL(N, 30)
        
    b_pol = bary_poly(tau, uu)
    coll_p_t = t_0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(coll_p_t, uu, color+'o', label = 'control '+scheme)
    #if N<=50:
    #    u_pol = lagrangePolynomial(tau, uu)
    #    u_interp_f = lambdify([x,],u_pol)
    #    plt.plot(t_int_arr, u_interp_f(tau_int_arr), color)
    #else:
    #    plt.plot(coll_p_t, uu, color+'--')
    plt.plot(t_int_arr, b_pol(tau_int_arr), color)

In [ ]:

plt.figure(figsize=[16,14])
N_Problem = 1

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
plt.title('Collocation Points:' + str(results_bloq['LG2']['N'][N_Problem]))
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plot_u(
        results_bloq[scheme]['u'][N_Problem],
        scheme,
        results_bloq[scheme]['t_f'][N_Problem],
        color
    )
plt.grid()
plt.legend()

In [ ]:
scheme = 'LGL'
_p_u = poly_u(
    results_bloq[scheme]['u'][1],
    scheme
)
_p_u

In [ ]:
#for ii in range(len(results_bloq[scheme]['N'])):
for ii in range(5):
    _n = results_bloq[scheme]['N'][ii]
    print(f'\n\nN = {_n}')
    for scheme in schemes:
        _p_u = poly_u(
            results_bloq[scheme]['u'][ii],
            scheme
        )
        print(f'\n\t{scheme}')
        for jj in range(_n-1, _n-5, -1):
            print(f'x^{jj}: {"{:.1e}".format(float(_p_u.coeff(x**jj)))}\t', end='')
f'{"{:.2e}".format(_p_u.coeff(x**4))}'

In [ ]:
def plot_q(qq, scheme, tf, color = 'k', t0 = 0, N_interp = 300):
    N = len(qq)
    if scheme == 'LG2':
        tau = LG2(N, 30)
    elif scheme == 'LGLm':
        tau = LGL(N, 30)
    elif scheme == 'LG':
        tau = LG_1(N, 30)
    else:
        tau = LGL(N, 30)
    int_p_t = t_0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(int_p_t, qq, color+'o', label = 'q '+scheme)
    if N<=50:
        q_pol = lagrangePolynomial(tau, qq)
        q_interp_f = lambdify([x,],q_pol)
        plt.plot(t_int_arr, q_interp_f(tau_int_arr), color)
    else:
        plt.plot(coll_p_t, qq, color+'--')
    return q_pol

In [ ]:

plt.figure(figsize=[16,14])
N_Problem = 2

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
plt.title('Collocation Points:' + str(results_bloq['LGL']['N'][N_Problem]))
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plot_q(
        results_bloq[scheme]['q'][N_Problem],
        scheme,
        results_bloq[scheme]['t_f'][N_Problem],
        color
    )
    print(scheme, ', len q:', len(results_bloq[scheme]['q'][N_Problem]))
plt.grid()
plt.legend()

In [ ]:

plt.figure(figsize=[16,14])
N_Problem = 0

schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
plt.title('Collocation Points:' + str(results_bloq['LGL']['N'][N_Problem]))
#for ii in range(5):
scheme = schemes[3]
_v_b = 2/(results_bloq[scheme]['t_f'][N_Problem]) *matrix_D_bary(results_bloq[scheme]['N'][N_Problem], scheme,20) @ results_bloq[scheme]['q'][N_Problem]
_pol_1 = plot_q(
    results_bloq[scheme]['v'][N_Problem],
    scheme,
    results_bloq[scheme]['t_f'][N_Problem],
    color
)
_pol_2 = plot_q(
    _v_b,
    scheme,
    results_bloq[scheme]['t_f'][N_Problem],
    'b'
)
plt.grid()
plt.legend()

In [ ]:
_pol_1

In [ ]:
_pol_2

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Hermite Interpolation Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_hsyi' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            #print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'Hermite')[2])
    
        results_bloq[scheme]['tr_err_hs'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_hs'], 'o' + color, alpha = 1)
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_hs'], color, label = scheme, alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_bloq['LGL']['N'], 10, 1e-3, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Hermite Interpolation Transcription Dinamics Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm',
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_d_hs' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            #print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'Hermite')[1])
        results_bloq[scheme]['tr_err_d_hs'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_d_hs'], 'o' + color, alpha = 1)
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_d_hs'], color, label = scheme, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_d_lin' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            #print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'lin')[1])
        results_bloq[scheme]['tr_err_d_lin'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_d_lin'], '--' + color, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_v'], '--' + color)
    
plt.grid(which='both')
plt.legend()
plt.yscale('log')
#plt.ylim(0,0.2)
#plt.vlines(results['LGL'][4], 2, 2e-28, 'k',':')

results_bloq['LG'].pop('tr_err_u_lin')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Lineal Interpolation for U Transcription Dynamics Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#, 'LGLm'
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_d_u_lin' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'pol')[1])
        results_bloq[scheme]['tr_err_d_u_lin'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_d_u_lin'], 'o' + color, alpha = 1)
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_d_u_lin'], color, label = scheme, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_d_lin' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'lin')[1])
        results_bloq[scheme]['tr_err_d_lin'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_d_lin'], '--' + color, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_v'], '--' + color)
    
plt.grid(which='both')
plt.legend()
plt.yscale('log')
#plt.ylim(0, 0.2)
#plt.vlines(results['LGL'][4], 2, 2e-28, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Lineal Interpolation for Q Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#, 'LGLm'
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_u_lin' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'pol')[2])
        results_bloq[scheme]['tr_err_u_lin'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_u_lin'], '--' + color, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_q_lin' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'pol',
                x_interp = 'lin')[2])
        results_bloq[scheme]['tr_err_q_lin'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_q_lin'], 'o' + color, alpha = 1)
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_q_lin'], color, label = scheme, alpha = 1)

for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_bloq[scheme]['N']
    if not 'tr_err_lin' in results_bloq[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_bloq[scheme]['q'][jj],
                results_bloq[scheme]['v'][jj],
                results_bloq[scheme]['u'][jj],
                scheme,
                0.,
                results_bloq[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'lin')[2])
        results_bloq[scheme]['tr_err_lin'] = lite
    plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err_lin'], ':' + color, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    #plt.plot(results_bloq[scheme]['N'], results_bloq[scheme]['tr_err'], '--' + color)
    
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.ylim(1e-4, 0.5)
#plt.vlines(results['LGL'][4], 2, 2e-28, 'k',':')

## Problema: Swing up del péndulo:

$$
\begin{split}\begin{array}{lc}
\begin{array}{l}
\text{minimize:} \\
x(\cdot) \in \mathbb{R}^2, \, u(\cdot) \in \mathbb{R}
\end{array}
\quad \displaystyle t_f
\\
\\
\text{subject to:} \\
\\
\begin{array}{ll}
\left\{
\begin{array}{l}
\dot{x}_0 = x_1 \\
\dot{x}_1 = u - sin(x_0) \\
-u_{max} \le u \le u_{max} , \quad
\end{array} \right. & \text{for} \, 0 \le t \le T \\
x_0(0)=0, \quad x_1(0)=0, x_0(T) = pi/2 , x_1(T) = 0
\end{array}
\end{array}\end{split}
$$
with $T=10$.

siendo $$x_0 = \theta$$ $$x_1 = \theta'$$

In [ ]:
@lru_cache
def obj_f_u_LGL_pend(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LGL(N, precission), coefs)
    res = integrate(cos(pol_lag), [x, -1, 1])
    return lambdify(coefs, res)

@lru_cache
def obj_f_u_LG2_pend(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG(N, precission), coefs)
    res = integrate(cos(pol_lag), [x, -1, 1])
    return lambdify(coefs, res)

In [ ]:
x_sym = cas.SX.sym('x', 2)
x_dot_sym = cas.SX.sym('x_dot', 2)
u_sym = cas.SX.sym('u',1)

dynam_f_x_pend = cas.Function(
    "dynamics_x",
    [x_sym, x_dot_sym, u_sym],
    [x_dot_sym[1]-(u_sym-cas.sin(x_sym[0]))]
)

In [ ]:
q_sym = cas.SX.sym('q', 1)
q_dot_sym = cas.SX.sym('q_dot', 1)
q_dot_dot_sym = cas.SX.sym('q_dot_dot', 1)
u_sym = cas.SX.sym('u',1)

dynam_f_q_pend = cas.Function(
    "dynamics_q",
    [q_sym, q_dot_sym, q_dot_dot_sym, u_sym],
    [q_dot_dot_sym-(u_sym - cas.sin(q_sym))]
)

In [ ]:
def plot_x(qq, scheme, tf, color = 'k', t0 = 0, N_interp = 300, label = 'q'):
    N = len(qq)
    tau = get_taus(N, scheme)
    b_pol = bary_poly(tau, qq)
    coll_p_t = t_0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(coll_p_t, qq, color+'o', label = label + ' ' +scheme)
    #if N<=50:
    #    q_pol = lagrangePolynomial(tau, qq)
    #    q_interp_f = lambdify([x,],q_pol)
    #    plt.plot(t_int_arr, q_interp_f(tau_int_arr), color)
    #else:
    #    plt.plot(coll_p_t, qq, color+'--')
    plt.plot(t_int_arr, b_pol(tau_int_arr), color)

In [ ]:
@lru_cache
def min_t_pend(col_points = 7, scheme = 'LGL', solve_repetitions = 20, precission = 20):
    
    opti, N, x_opti, u_opti, tau_arr = opti_setup(col_points, scheme, precission)
    
    t_f = opti.variable(1)
    t_0 = 0
    u_lim = opti.parameter(1)
    
    D_mat = sympy2casadi(matrix_D_bary(N-1, scheme,precission), [], [])
    x_dot_opti = 2/(t_f - t_0) * D_mat@x_opti
    x_dot_dot_opti = 2/(t_f - t_0) * D_mat@x_dot_opti

    cost = t_f
    opti.minimize(cost)
         
    opti.subject_to(t_f > 0)
    opti_start_point_constr(opti, scheme, x_opti, x_dot_opti, x_s = [0,0])
    opti_end_point_constr(opti, scheme, x_opti, x_dot_opti, N, x_f = [np.pi/2,0])
    opti_col_point_constr(opti, scheme, x_opti, x_dot_opti, x_dot_dot_opti, u_opti, 
                          dynam_f_x_pend, dynam_f_q_pend, N)
    
    for ii in range(col_points):
        opti.subject_to(u_opti[ii,:] <= u_lim[:])
        opti.subject_to(-u_lim[:] <= u_opti[ii,:])

    opti.set_value(u_lim, 0.5)
    opti.set_initial(t_f, 10.)
    
    #opti.set_initial(u_opti, list(np.linspace(-1,1,col_points)))
    
    sol, cpudt, err_count = chrono_solve_random_init(opti, solve_repetitions, u_opti, u_lim)
    
    t_f_sol = sol.value(t_f)
    
    qq_sol, vv_sol, uu_sol = opti_read_sol_arrays(scheme, sol, x_opti, u_opti, D_mat, t_f_sol)
        
    tran_err_q, tran_err_v, tran_err = error_transcr(qq_sol, vv_sol, uu_sol, scheme, t_0, t_f_sol,
                             u_interp='pol', x_interp='pol', problem='pend',)
    
    return t_f_sol, cpudt, uu_sol, qq_sol, vv_sol, tran_err_q, tran_err_v, tran_err, err_count

In [ ]:
if not 'results_pend' in globals():
    results_pend = {}
test_N = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,30,35,40,45,50]
#test_N = [6,7]#,8,9,10,11,12,13]
for scheme in ['LG','LGL', 'D2', 'LG2']:#,'LGLm'
    print(f'\n\t\t NUEVO ESQUEMA: {scheme}\n')
    
    t_f_list = []
    cpudt_list = []
    uu_sol_list = []
    qq_sol_list = []
    vv_sol_list = []
    tran_err_q_list = []
    tran_err_v_list = []
    tran_err_list = []
    err_count_list = []
    for N_col in test_N:
        print(f' Problema: {N_col} Collocation Points')
        time.sleep(0.01)
        t_f_sol, cpudt, uu_sol, qq_sol, vv_sol, tran_err_q, tran_err_v, tran_err, err_count = min_t_pend(col_points = N_col,
                                              scheme = scheme,
                                              solve_repetitions = 2000,
                                              precission = 20)
        t_f_list.append(t_f_sol)
        cpudt_list.append(cpudt)
        uu_sol_list.append(uu_sol)
        qq_sol_list.append(qq_sol)
        vv_sol_list.append(vv_sol)
        tran_err_q_list.append(tran_err_q)
        tran_err_v_list.append(tran_err_v)
        tran_err_list.append(tran_err)
        err_count_list.append(err_count)
    results_pend[scheme] = {
        't_f': t_f_list ,
        'cpu_t' : cpudt_list ,
        'u' : uu_sol_list,
        'N' : test_N,
        'q' : qq_sol_list,
        'v' : vv_sol_list,
        'tr_err_q' : tran_err_q_list,
        'tr_err_v' : tran_err_v_list,
        'tr_err' : tran_err_list,
        'err_count' : err_count_list,}

In [ ]:
plt.figure(figsize=[16,14])
N_Problem = -1
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
ii = 3
scheme = schemes[ii]
plt.title('Collocation Points:' + str(results_pend[scheme]['N'][N_Problem]) + ' ' + scheme)
color = colours[ii]
plot_u(
    results_pend[scheme]['u'][N_Problem],
    scheme,
    t_f_sol,
    'g',
    N_interp=1000
)
plot_x(
    results_pend[scheme]['q'][N_Problem],
    scheme,
    t_f_sol,
    'r'
)
plot_x(
    results_pend[scheme]['v'][N_Problem],
    scheme,
    t_f_sol,
    'b',
    label = 'v'
)
plt.grid()
plt.legend()

In [ ]:
plt.figure(figsize=[16,14])


plt.title('t_f')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    quality = results_pend[scheme]['t_f']#[np.mean(np.cos(ii)) for ii in results_pend[scheme][4]]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], quality, color + 'o', label = scheme, alpha = 1)
    plt.plot(results_pend[scheme]['N'], quality, color, alpha = 1)
plt.grid()
plt.legend()
plt.vlines(results_pend['LGL']['N'], 4.7, 6, 'k',':')
#plt.ylim([1.92,2.15])

In [ ]:
#for ii in range(len(results_pend[scheme]['N'])):
for ii in range(5):
    _n = results_pend[scheme]['N'][ii]
    print(f'\n\nN = {_n}')
    for scheme in schemes:
        _p_u = poly_u(
            results_pend[scheme]['u'][ii],
            scheme
        )
        print(f'\n\t{scheme}')
        for jj in range(_n-1, _n-5, -1):
            print(f'x^{jj}: {"{:.1e}".format(float(_p_u.coeff(x**jj)))}\t', end='')
f'{"{:.2e}".format(_p_u.coeff(x**4))}'

In [ ]:

plt.figure(figsize=[16,14])
N_Problem = 0

plt.title('Collocation Points:' + str(test_N[N_Problem]))
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plot_u(
        results_pend[scheme]['u'][N_Problem],
        scheme,
        results_pend[scheme]['t_f'][N_Problem],
        color
    )
plt.grid()
plt.legend()

In [ ]:

plt.figure(figsize=[16,14])
N_Problem = 0

plt.title('Collocation Points:' + str(test_N[N_Problem]))
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plot_q(
        results_pend[scheme]['q'][N_Problem],
        scheme,
        results_pend[scheme]['t_f'][N_Problem],
        color
    )
    
plt.grid()
plt.legend()

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    #plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], color, label = scheme, alpha = 1)
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_q'], ':' + color, label = scheme + ' q', alpha = 1)
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_v'], '-.' + color, label = scheme + ' d', alpha = 1)
plt.grid()
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1e-2, 1e-13, 'k',':')
plt.ylim(1e-13, 1e-2)

In [ ]:
solve_repetitions = 2000
for scheme in schemes:
    print('Runtime errors', scheme, ':', np.sum(results_pend[scheme]['err_count']), 'errores en', solve_repetitions, 'iteraciones')
plt.figure(figsize=[16,14])
plt.title('Error fraction')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], np.array(results_pend[scheme]['err_count'])/solve_repetitions, color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_pend['LGL']['N'], 0, 0.2, 'k',':')
#plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])
plt.title('Casadi mean CPU time, random u init, 2000 runs')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_pend['LGL']['N'], 0, 0.22, 'k',':')
#plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])
plt.title('CPU time')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_pend['LGL']['N'], 0, 0.22, 'k',':')
#plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])
plt.title('CPU time')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['cpu_t'], color, label = scheme)
plt.grid()
plt.legend()
plt.vlines(results_pend['LGL']['N'], 0, 0.22, 'k',':')
#plt.yscale('log')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Lineal Interpolation Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        #print('scheme: ', scheme, ' N: ', test_N[jj])
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'lin',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], lite, color, label = scheme, alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 1.4e-2, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Lineal Interpolation Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        #print('scheme: ', scheme, ' N: ', test_N[jj])
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'lin',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], lite, color, label = scheme, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], color + '--', label = scheme, alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 3e-13, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Hermite Interpolation Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'Hermite',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], lite, color, label = scheme, alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 1e-3, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Hermite Interpolation Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'Hermite',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], lite, color, label = scheme, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'lin',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, '--' + color, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], color + '--', alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 2e-13, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Hermite Interpolation Transcription Error with smoothed u')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'smooth',
            x_interp = 'Hermite',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], lite, color, label = scheme, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    lite = []
    test_N = results_pend[scheme]['N']
    for jj in range(len(test_N)):
        lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'Hermite',
            problem='pend')[2])
    
    plt.plot(results_pend[scheme]['N'], lite, ':' + color, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], color + '--', alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 2e-13, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Lineal Interpolation for u Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#'LGLm', 
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_pend[scheme]['N']
    if not 'tr_err_u_lin' in results_pend[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'pol',
            problem='pend')[2])
        results_pend[scheme]['tr_err_u_lin'] = lite
    
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_u_lin'], 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_u_lin'], color, label = scheme, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_pend[scheme]['N']
    
    if not 'tr_err_lin' in results_pend[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            #print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
            results_pend[scheme]['q'][jj],
            results_pend[scheme]['v'][jj],
            results_pend[scheme]['u'][jj],
            scheme,
            0.,
            results_pend[scheme]['t_f'][jj],
            u_interp = 'lin',
            x_interp = 'lin',
            problem='pend')[2])
        results_pend[scheme]['tr_err_lin'] = lite
    
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_lin'], '--' + color, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], color + '--', alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 3e-13, 'k',':')

In [ ]:
plt.figure(figsize=[16,14])

plt.title('Lineal Interpolation for u Transcription Error')
schemes = ['LGL', 'D2', 'LG2', 'LG']#, 'LGLm'
colours = ['r', 'b', 'g', 'y', 'm']
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_pend[scheme]['N']
    if not 'tr_err_q_lin' in results_pend[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_pend[scheme]['q'][jj],
                results_pend[scheme]['v'][jj],
                results_pend[scheme]['u'][jj],
                scheme,
                0.,
                results_pend[scheme]['t_f'][jj],
                u_interp = 'pol',
                x_interp = 'lin',
                problem='pend')[2])
        results_pend[scheme]['tr_err_q_lin'] = lite
    
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_q_lin'], 'o' + color, alpha = 1)
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_q_lin'], color, label = scheme, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_pend[scheme]['N']
    if not 'tr_err_u_lin' in results_pend[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            #print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_pend[scheme]['q'][jj],
                results_pend[scheme]['v'][jj],
                results_pend[scheme]['u'][jj],
                scheme,
                0.,
                results_pend[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'pol',
                problem='pend')[2])
        results_pend[scheme]['tr_err_u_lin'] = lite
    
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_u_lin'], '-.' + color, alpha = 1)
    
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    test_N = results_pend[scheme]['N']
    
    if not 'tr_err_lin' in results_pend[scheme].keys():
        lite = []
        for jj in range(len(test_N)):
            print('scheme: ', scheme, ' N: ', test_N[jj])
            lite.append(error_transcr(
                results_pend[scheme]['q'][jj],
                results_pend[scheme]['v'][jj],
                results_pend[scheme]['u'][jj],
                scheme,
                0.,
                results_pend[scheme]['t_f'][jj],
                u_interp = 'lin',
                x_interp = 'lin',
                problem='pend')[2])
        results_pend[scheme]['tr_err_lin'] = lite
    
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err_lin'], ':' + color, alpha = 1)
for ii in range(len(schemes)):
    scheme = schemes[ii]
    color = colours[ii]
    plt.plot(results_pend[scheme]['N'], results_pend[scheme]['tr_err'], color + '--', alpha = 1)
plt.grid(which='both')
plt.legend()
plt.yscale('log')
plt.vlines(results_pend['LGL']['N'], 1, 3e-13, 'k',':')